# Load Processed Data into Vector Database

This notebook loads output from data prep kit into Milvus

**Step-4 in this workflow**

![](../media/rag-overview-2.png)


## Configuration

In [1]:
class MyConfig:
    pass
MY_CONFIG = MyConfig()

MY_CONFIG.INPUT_DATA_DIR = 'data/granite-docs/output_final/'
# MY_CONFIG.INPUT_DATA_REMOTE = "https://github.com/sujee/data-prep-kit-examples/blob/main/requirements.txt"

MY_CONFIG.DB_URI = './rag_demo_dataprepkit_1.db'  # For embedded instance
#MY_CONFIG.DB_URI = 'http://localhost:19530'  # For Docker instance
MY_CONFIG.COLLECTION_NAME = 'dataprepkit_granite_docs'

## Step-: Load Parquet Data

Load all  `.parquet` files in the given dir

In [2]:
import pandas as pd
import glob

print ('Loading data from : ', MY_CONFIG.INPUT_DATA_DIR)

# Get a list of all Parquet files in the directory
parquet_files = glob.glob(f'{MY_CONFIG.INPUT_DATA_DIR}/*.parquet')
print ("Number of parquet files to read : ", len(parquet_files))
print ()

# Create an empty list to store the DataFrames
dfs = []

# Loop through each Parquet file and read it into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    print (f"Read file: '{file}'.  number of rows = {df.shape[0]}")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data_df = pd.concat(dfs, ignore_index=True)

print (f"\nTotal number of rows = {data_df.shape[0]}")

Loading data from :  data/granite-docs/output_final/
Number of parquet files to read :  1

Read file: 'data/granite-docs/output_final/Granite_Foundation_Models.parquet'.  number of rows = 216

Total number of rows = 216


In [3]:

## Shape the data

MY_CONFIG.EMBEDDING_LENGTH =  len(data_df.iloc[0]['embeddings'])
print ('embedding length: ', MY_CONFIG.EMBEDDING_LENGTH)

# rename 'embeddings' columns as 'vector' to match default schema
# if 'vector' not in data_df.columns and 'embeddings' in data_df.columns:
#     data_df = data_df.rename( columns= {'embeddings' : 'vector'})
# if 'text' not in data_df.columns and 'contents' in data_df.columns:
#     data_df = data_df.rename( columns= {'contents' : 'text'})

data_df = data_df.rename( columns= {'embeddings' : 'vector', 'contents' : 'text'})

print (data_df.info())
data_df.head(3)

embedding length:  384
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   filename                      216 non-null    object 
 1   num_pages                     216 non-null    int64  
 2   num_tables                    216 non-null    int64  
 3   num_doc_elements              216 non-null    int64  
 4   document_id                   216 non-null    object 
 5   ext                           216 non-null    object 
 6   hash                          216 non-null    object 
 7   size                          216 non-null    int64  
 8   date_acquired                 216 non-null    object 
 9   pdf_convert_time              216 non-null    float64
 10  source_filename               216 non-null    object 
 11  text                          216 non-null    object 
 12  doc_jsonpath                  216 non-nul

,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words,vector
0,Granite_Foundation_Models.pdf,20,13,445,6b7b64a5-e18c-4dff-8ade-8b5703db74e1,pdf,a30250ea1dbc6510350d81860e86748e99709566030a50...,357701,2024-08-06T21:56:44.353184,70.621034,...,0.0,2,0.0,0.0,False,0.0,0.0,1.0,True,"[-0.007855933, 0.0186794, 0.04243699, -0.01503..."
1,Granite_Foundation_Models.pdf,20,13,445,6b7b64a5-e18c-4dff-8ade-8b5703db74e1,pdf,a30250ea1dbc6510350d81860e86748e99709566030a50...,357701,2024-08-06T21:56:44.353184,70.621034,...,0.0,1,0.0,0.0,False,0.0,0.0,1.0,False,"[-0.003576741, 0.009818693, 0.03441964, -0.006..."
2,Granite_Foundation_Models.pdf,20,13,445,6b7b64a5-e18c-4dff-8ade-8b5703db74e1,pdf,a30250ea1dbc6510350d81860e86748e99709566030a50...,357701,2024-08-06T21:56:44.353184,70.621034,...,0.0,6,0.0,0.0,False,0.0,0.0,1.0,True,"[-0.022207905, 0.0050711543, 0.022928601, -0.0..."


## Connect to Vector Database

Milvus can be embedded and easy to use.


In [4]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_demo_dataprepkit_1.db


# Create A Collection



In [5]:
# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name=MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


milvus_client.create_collection(
    collection_name=MY_CONFIG.COLLECTION_NAME,
    dimension=MY_CONFIG.EMBEDDING_LENGTH,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    auto_id=True
)
print ("✅ Created collection :", MY_CONFIG.COLLECTION_NAME)


✅ Created collection : dataprepkit_granite_docs


In [6]:
res = milvus_client.insert(collection_name=MY_CONFIG.COLLECTION_NAME, data=data_df.to_dict('records'))

print('inserted # rows', res['insert_count'])

milvus_client.get_collection_stats(MY_CONFIG.COLLECTION_NAME)

inserted # rows 216


{'row_count': 216}

## Close DB Connection

Close the connection so the lock files are relinquished and other notebooks can access the db

In [7]:
milvus_client.close()

## Test your data by doing a Vector Search

See notebook [vector_search.ipynb](vector_search.ipynb)